Transfer Learning Notebook 

Martin Jasinski
2025

In [27]:
import torch
import torch.nn as nn
import torchvision

import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt


In [28]:
# -------------------------------------------------------------------------------
# Import ResNet from scratch
# -------------------------------------------------------------------------------

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(BasicBlock, 64, 2, stride=1)
        self.layer2 = self._make_layer(BasicBlock, 128, 2, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 512, 2, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [29]:
# -------------------------------------------------------------------------------
# Create manual training
# -------------------------------------------------------------------------------

# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 (adjust as needed)
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])

# Define dataset directory
data_dir = "Dataset_1/Train"  # Replace with your dataset path

# Load dataset using ImageFolder (assumes subfolders are class labels)
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# check if the dataset is properly imported
print("Class-to-index mapping:", dataset.class_to_idx)
print("Number of images in dataset:", len(dataset))


Class-to-index mapping: {'BRCA': 0, 'GBM': 1}
Number of images in dataset: 1000


In [31]:
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
 
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:{torch.cuda.current_device()}")
       
print(f"Name of current CUDA device:{torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system? False
CUDA version: None


AssertionError: Torch not compiled with CUDA enabled

In [23]:
# -------------------------------------------------------------------------------
# Training Loop
# -------------------------------------------------------------------------------

# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the model and move it to GPU
model = ResNet18().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Define DataLoader ( random set of training from the training set )
trainloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)

loss_total = []
for epoch in range(50):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch} loss: {running_loss / len(trainloader)}')
    loss_total.append(running_loss / len(trainloader))

Using device: cpu


KeyboardInterrupt: 

In [ ]:
# -------------------------------------------------------------------------------
# Plot the final resulting loss
# -------------------------------------------------------------------------------

fig = plt.figure(figsize = (12,12))
ax = fig.add_subplot(111)
ax.plot(np.arange(0,50,1), loss_total, label="Running Loss")
ax.title("Training Loss vs. Epoch")
ax.set_xlabel("Epoch")
ax.set_ylabel("Cross Entropy Loss $L$")
ax.grid()
plt.show()